In [ ]:
'''
exploratory notebook containing
data cleaning and base pipeline model
for application

DCohen
MFa
EPadilla
RJProctor
'''

In [2]:
!pip install category_encoders==2.*

     |████████████████████████████████| 80 kB 872 kB/s 
     |████████████████████████████████| 25.8 MB 4.5 MB/s 
     |████████████████████████████████| 6.8 MB 4.0 MB/s 
     |████████████████████████████████| 231 kB 2.9 MB/s 
     |████████████████████████████████| 9.4 MB 414 kB/s 
     |████████████████████████████████| 301 kB 13.1 MB/s 


In [4]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

print(f'matplotlib=={matplotlib.__version__}')
print(f'seaborn=={sns.__version__}')
print(f'joblib=={joblib.__version__}')
print(f'pandas=={pd.__version__}')
print(f'numpy=={np.__version__}')
print(f'category_encoders=={category_encoders.__version__}')
print(f'sklearn=={sklearn.__version__}')
# matplotlib==3.2.2
# seaborn==0.11.0
# joblib==0.16.0
# pandas==1.1.2
# numpy==1.18.5
# category_encoders==2.2.2
# sklearn==0.22.2.post1

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Need to download the AirBnB training data to you local machine from Kaggle
# https://www.kaggle.com/rudymizrahi/airbnb-listings-in-major-us-cities-deloitte-ml

df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()


,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,2015-04-19,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,t,100%,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74111 entries, 0 to 74110
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      74111 non-null  int64  
 1   log_price               74111 non-null  float64
 2   property_type           74111 non-null  object 
 3   room_type               74111 non-null  object 
 4   amenities               74111 non-null  object 
 5   accommodates            74111 non-null  int64  
 6   bathrooms               73911 non-null  float64
 7   bed_type                74111 non-null  object 
 8   cancellation_policy     74111 non-null  object 
 9   cleaning_fee            74111 non-null  bool   
 10  city                    74111 non-null  object 
 11  description             74111 non-null  object 
 12  first_review            58247 non-null  object 
 13  host_has_profile_pic    73923 non-null  object 
 14  host_identity_verified  73923 non-null

In [ ]:
def wrangle(X):
  # make a copy
  X = X.copy()

  # columns with unusable variance
  unusable_variance = ['zipcode']

  # columns with high percentage of missing values
  high_nans = ['first_review','host_response_rate','last_review',
               'review_scores_rating','thumbnail_url']

  # categorical values with high cardinality
  # 'neighborhood' has 620 and 'thumbnail_url' has many thousands
  high_card = ['neighbourhood','thumbnail_url','name','amenities',
               'description', 'id']

  # Get the price and drop the log of price
  X['int_price'] = np.exp(X['log_price'])
  X = X.drop(['log_price'] + unusable_variance + high_nans + high_card, axis=1)

  return X

In [ ]:
df = wrangle(df)

In [ ]:
df.head()

,property_type,room_type,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,host_has_profile_pic,host_identity_verified,host_since,instant_bookable,latitude,longitude,number_of_reviews,bedrooms,beds,int_price
0,Apartment,Entire home/apt,3,1.0,Real Bed,strict,True,NYC,t,t,2012-03-26,f,40.696524,-73.991617,2,1.0,1.0,150.0
1,Apartment,Entire home/apt,7,1.0,Real Bed,strict,True,NYC,t,f,2017-06-19,t,40.766115,-73.989040,6,3.0,3.0,169.0
2,Apartment,Entire home/apt,5,1.0,Real Bed,moderate,True,NYC,t,t,2016-10-25,t,40.808110,-73.943756,10,1.0,3.0,145.0
3,House,Entire home/apt,4,1.0,Real Bed,flexible,True,SF,t,t,2015-04-19,f,37.772004,-122.431619,0,2.0,2.0,750.0
4,Apartment,Entire home/apt,2,1.0,Real Bed,moderate,True,DC,t,t,2015-03-01,t,38.925627,-77.034596,4,0.0,1.0,115.0


In [ ]:
target = 'int_price'
y = df[target]

X = df.drop('int_price', axis=1)

In [ ]:
X.shape

(74111, 17)

In [ ]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    StandardScaler(),
    LassoCV()
)

model.fit(X, y)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['property_type', 'room_type', 'bed_type',
                                     'cancellation_policy', 'city',
                                     'host_has_profile_pic',
                                     'host_identity_verified', 'host_since',
                                     'instant_bookable'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', return_df=True,
                               use_cat_names=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregression',
               

In [ ]:
# Check metrics

print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model.predict(X_val)))

In [ ]:
# check metrics

print('Training R-score:', r2_score(y_train, model.predict(X_train)))
print('Validation R-score:', r2_score(y_val, model.predict(X_val)))

In [ ]:
from joblib import dump
pipeline = model
dump(pipeline, 'pipeline.joblib', compress=True)
